## 4.1画像データ

In [4]:
from PIL import Image
import numpy as np
import torch

In [2]:
PATH = "../deep-learning-with-pytorch-ja-main/data/p1ch2/bobby.jpg"
img = Image.open(PATH)

In [3]:
img_arr = np.array(img)
print(img_arr.shape)

(720, 1280, 3)


In [20]:
#permuteは次元の位置を変更する

#tensorflowはチャネルが4
#torchは2
#batchはどちらも最初
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(img.shape, out.shape)

torch.Size([720, 1280, 3]) torch.Size([3, 720, 1280])


In [7]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
#カメラの写真のフォーマット8ビットのint型

In [8]:
import os
data_dir = "../deep-learning-with-pytorch-ja-main/data/p1ch4/image-cats"
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] =='.png']

#os.path.splitext(ファイル名)で拡張子を取得

In [9]:
#batchに画像データ三つを上書き
for i, filename in enumerate(filenames):
    img_arr = Image.open(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(np.array(img_arr))
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

In [40]:
batch.shape, batch[2][0][0][0]

(torch.Size([3, 3, 256, 256]), tensor(238, dtype=torch.uint8))

In [41]:
#データの正規化

#ニューラルネットでは0から1、または-1から1の範囲で性能が高い
#画像データは要素を浮動小数点にして、正規化するのが普通

#8bit符号なし表現なら最大値の255で割る手法がある
batch = batch.float()
batch /= 255.0

In [48]:
batch.shape[1], batch.shape

(3, torch.Size([3, 3, 256, 256]))

In [58]:
batch.shape, batch[:,1].shape, batch[:,1,1].shape

(torch.Size([3, 3, 256, 256]), torch.Size([3, 256, 256]), torch.Size([3, 256]))

In [61]:
batch[:,1,:,:][0][0][0], batch[:,1][0][0][0]

(tensor(0.4632), tensor(0.4632))

In [47]:
#もう一つの正規化手法は平均0、標準偏差1にする標準化
n_chanels = batch.shape[1]
for c in range(n_chanels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean ) / std

## 4.2三次元データ（CT)

In [62]:
#次元はバッチ、チャネル、深さ、たて、横の五次元になる

In [16]:
import imageio
dir_path = "../deep-learning-with-pytorch-ja-main/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr = imageio.volread(dir_path, "DICOM")
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%69/99 files (69.7%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 69/99  (69.799/99  (100.0%)


(99, 512, 512)

In [17]:
vol = torch.from_numpy(vol_arr).float()
vol.shape

torch.Size([99, 512, 512])

In [18]:
#バッチ方向がない四次元
vol = torch.unsqueeze(vol, 0)
vol.shape

torch.Size([1, 99, 512, 512])

##   4.3表形式のデータ

In [55]:
import csv

In [65]:
#wine_path = "../deep-learning-with-pytorch-ja-main/data/p1ch4/tabular-wine/winequality-white.csv"
wine_path = "../deep-learning-with-pytorch-ja/data/p1ch4/tabular-wine/winequality-white.csv"
#1行目はカラム名のためスキップして読み込み
wine_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
wine_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [66]:
col_list = next(csv.reader(open(wine_path), delimiter=";"))
wine_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [67]:
wineq = torch.from_numpy(wine_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [68]:
#連続値、順序尺度、カテゴリカル変数には注意

In [69]:
#スコアの表現

#スコアはラベルとして扱う分類タスクか、連続値として回帰タスクか

In [70]:
#最後の列が目的変数のためそれを除く
data = wineq[:, :-1]
print(data.shape)

target = wineq[:, -1]
print(target.shape)

torch.Size([4898, 11])
torch.Size([4898])


In [71]:
target.dtype

torch.float32

In [72]:
#ターゲットがワインの色名のような文字列のラベルであれば、文字列を整数値にして同様にアプローチできる
target = wineq[:, -1].long()
target.dtype

torch.int64

In [76]:
target.shape[0]

4898

In [96]:
#ワンホットエンコーディング

#データ数とラベル数のテンソル作成
target_onehot = torch.zeros(target.shape[0], 10)
print(target_onehot.shape, target.shape,  target.unsqueeze(1).shape, target.unsqueeze(1))

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

torch.Size([4898, 10]) torch.Size([4898]) torch.Size([4898, 1]) tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [97]:
#scatter_で最後が_で終わるのは内部でテンソルを上書きする
#引数は
#one_hotにする配列 ０が行、１が列
#one_hotにしたい列（ラベル）　このindexはone_ohtで表現するテンソルと同じ次元にする必要がありunsqueeze(1)
#入力させる値

In [98]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [99]:
#テンソルの計算
print(data.shape)
data_mean = torch.mean(data, dim=0)
data_mean.shape

torch.Size([4898, 11])


torch.Size([11])

In [100]:
data_var = torch.var(data, dim=0)
data_var.shape

torch.Size([11])

In [101]:
#正規化（標準化）
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized[1], data[1]

(tensor([-0.6574,  0.2159,  0.0480, -0.9447,  0.1477, -1.2529, -0.1497, -0.0091,
          0.7400,  0.0013, -0.8242]),
 tensor([6.3000e+00, 3.0000e-01, 3.4000e-01, 1.6000e+00, 4.9000e-02, 1.4000e+01,
         1.3200e+02, 9.9400e-01, 3.3000e+00, 4.9000e-01, 9.5000e+00]))

In [102]:
#閾値の設定
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [103]:
#bad_indexesの要素のうち20個がTure
#advanced indexing機能を使い、torch.boolのテンソルを利用し、 インデックスをつける

In [104]:
bad_data = data[bad_indexes]
bad_data.shape
#bad_dataテンソルは20行
#条件にTrueで返した行
#列は11列全て保有

torch.Size([20, 11])

In [105]:
#上の方法を使いワインの状態を三段階に分ける
bad_data = data[target <= 3]
mid_data = data[(target >3) & ( target <= 7)]
good_data = data[target >= 7]

In [106]:
bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data ,dim=0)
#3段階のワインの詳細データの表示
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print("{:2} {:20} {:6.2f} {:6.2f} {:6.2f}".format(i,*args))

 0 fixed acidity          7.60   6.86   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.33   0.33
 3 residual sugar         6.39   6.42   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.18  34.55
 6 total sulfur dioxide 170.60 138.70 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.19   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.47  11.42


In [107]:
#total sulfur dioxide(二酸化硫黄)が多いと質が悪いのか？
#閾値を作成し
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
total_sulfur_data

tensor([170., 132.,  97.,  ..., 111., 110.,  98.])

In [108]:
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()
# 第一引数が元data, 第二引数以下のテンソルがTure、大きい要素はFalse

(torch.Size([4898]), torch.bool, tensor(2727))

In [109]:
#数を確認すると半分以上は良質なワインか？
#targetを確認する
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [110]:
#数は合わない。
#重なりを確認する
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
print(n_matches)
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
print(n_matches / n_actual, n_matches / n_predicted)

2018
0.6193984039287906 0.74000733406674


## 4.4時系列データ

In [134]:
bike_path = "../deep-learning-with-pytorch-ja/data/p1ch4/bike-sharing-dataset/hour-fixed.csv"
bikes_numpy = np.loadtxt(bike_path, 
                      dtype=np.float32, 
                      delimiter=",", 
                      skiprows=1, 
                      converters={1: lambda x:float(x[8:10])}#日だけにしている,strだと読み込めない
                     )

In [155]:
#index、日、季節、年、月、時間、休日か、曜日、平日か、曜日、気象状況、温度
#体感温度、湿度、風速、カジュアルユーザ数、登録ユーザ数、レンタル自転車数
#1時間ごとのデータ
bikes = torch.from_numpy(bikes_numpy)
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [156]:
#ある単位量でのシーケンス処理をする
#単位：Cの次元：Nの時間軸のデータに変換してNN

In [157]:
#時間帯別に整形

In [158]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()
#bikesの列数を2次元に、1次元は24で日ごと、0次元は-1で自動計算

(torch.Size([730, 24, 17]), (408, 17, 1))

In [159]:
#日付の次元に沿って進むには24×17個の要素文進まないといけない
#N：730個のシーケンスを、C：17チャネル、L:24時間のデータでもつ形に変更
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [160]:
#テンソルはストレージに連続に格納されており、形状をviewで自由に変更できる
daily_bikes[1].shape

torch.Size([17, 24])

In [161]:
#時間の次元（１）がひとつ進むと、ストレージ内の要素が17個すすむことがわかる

In [162]:
first_day = bikes[:24]
first_day.dtype

torch.float32

In [163]:
#整数にします
first_day = bikes[:24].long()
first_day.dtype

torch.int64

訓練の準備 onehot

In [177]:
#訓練の準備

#初日のデータの確認
first_day = bikes[:24].long()
print(first_day.shape)
#天気は四種類
weather_onehot = torch.zeros(first_day.shape[0], 4)
print(weather_onehot.shape)

torch.Size([24, 17])
torch.Size([24, 4])


In [165]:
#天気
print(first_day[:, 9])
#1から４を、０から３に変更
print(first_day[:, 9].unsqueeze(1).long() -1)

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])
tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [1],
        [1],
        [1],
        [1]])


In [166]:
print(f"one_hotで {first_day.shape} -> 9番目の{weather_onehot.shape} 　最後にconcat ")

one_hotで torch.Size([24, 17]) -> 9番目のtorch.Size([24, 4]) 　最後にconcat 


In [167]:
weather_onehot.scatter_(
    dim =1,#one_hotにしたいカラム
    index=first_day[:, 9].unsqueeze(1).long() -1 ,#one_hotにしたい列（ラベル）　このindexはone_ohtで表現するテンソルと同じ次元にする
    value=1.0#入れる値)
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [168]:
#元データに結合
#:24 は一日のデータだから
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [169]:
#全データに適応（B,C,L)　B＝730　L=24　L=17

daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
                                   daily_bikes.shape[2])

In [172]:
#チャネルの24の代わりにラベル数の次元
daily_weather_onehot.shape, daily_bikes[:, 9, :].long().shape

(torch.Size([730, 4, 24]), torch.Size([730, 24]))

In [174]:
daily_weather_onehot.scatter_(1, 
                              daily_bikes[:, 9, :].long().unsqueeze(1) -1,
                              1.0
                             )

daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [176]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

連続変数に変換

天気は0.0から1.0の連続変数として扱うこともできる

In [179]:
#4までだから１引いて３で割る
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] -1.0) /3.0

In [180]:
#0,1スケーリング
temp = daily_bikes[:, 10, :]
temp_min =torch.min(temp)
temp_max =torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)/
                         (temp_max - temp_min))

In [182]:
#あるいは標準化
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, : ] - torch.mean(temp))/
                         torch.std(temp))

##　テキストデータ

In [ ]:
#テキストから数値へ変換